In [5]:
import torch
import sys

sys.path.append('../')
from src.models.components.xlsr_conformertcm_baseline import Model as XLSRConformerTCM

/home/hungdx/miniconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX


In [ ]:


ckpt = torch.load(
    '/nvme1/hungdx/Lightning-hydra/logs/train/runs/2024-12-14_08-35-06-large-corpus-conf-1/checkpoints/averaged_top5.ckpt')


/tmp/ipykernel_2523835/3872458690.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(


{'net.front_end.model.mask_emb': tensor([ 0.3542, -0.0494,  0.1383,  ...,  0.1020, -0.0372, -0.3062]), 'net.front_end.model.feature_extractor.conv_layers.0.0.weight': tensor([[[ 0.0287, -0.0233,  0.0084,  ...,  0.0542,  0.0124, -0.0191]],

        [[-0.1196,  0.1214,  0.0397,  ...,  0.1046, -0.2087,  0.0914]],

        [[-0.0354,  0.1245, -0.1888,  ...,  0.1807, -0.1008,  0.0313]],

        ...,

        [[-0.0453,  0.1214, -0.1294,  ..., -0.0071, -0.0086,  0.0079]],

        [[ 0.0006, -0.0174,  0.0571,  ..., -0.1079,  0.0002,  0.0241]],

        [[ 0.0377, -0.0839,  0.0504,  ..., -0.0567, -0.0318,  0.0342]]]), 'net.front_end.model.feature_extractor.conv_layers.0.0.bias': tensor([-0.0170, -0.0176, -0.0276, -0.0557, -0.0177, -0.0140, -0.0173, -0.1371,
        -0.0179, -0.0444, -0.0177, -0.0172, -0.0394, -0.0172, -0.0304, -0.0431,
        -0.0177, -0.0168, -0.0175, -0.0172, -0.0173, -0.0175, -0.0172, -0.0178,
        -0.0307, -0.0169, -0.0175, -0.0460, -0.0174, -0.0136, -0.0172, -0.0485

In [ ]:
args = ckpt['hyper_parameters']['args']['conformer']

In [ ]:


model = XLSRConformerTCM(args=args,
                         ssl_pretrained_path='/data/hungdx/asvspoof5/model/pretrained/xlsr2_300m.pt')

model

In [ ]:
from collections import OrderedDict


def load_model_weights(model, checkpoint, required_prefix='net'):
    """
    Load model weights from checkpoint with specific key prefix requirements.
    
    Args:
        model: PyTorch model to load weights into
        checkpoint_path: Path to the checkpoint file
        required_prefix: Required prefix for state dict keys (default: 'net')
        
    Returns:
        model: Model with loaded weights
    """
  
    state_dict = checkpoint if isinstance(
        checkpoint, dict) else checkpoint.state_dict()
    
    print(f"Loading {len(state_dict)} parameters from checkpoint")

    # Create new state dict with processed keys
    new_state_dict = OrderedDict()

    # Process the state dict keys
    for key, value in state_dict.items():
        # Skip keys that don't start with the required prefix
        if not key.startswith(required_prefix):
            continue

        # Remove the 'net.' prefix to match model's state dict keys
        new_key = key[len(required_prefix) + 1:]  # +1 for the dot after prefix
        new_state_dict[new_key] = value

    # Load the processed state dict
    try:
        model.load_state_dict(new_state_dict, strict=True)
        print(f"Successfully loaded {len(new_state_dict)} parameters")
    except RuntimeError as e:
        print(f"Error loading state dict: {str(e)}")
        raise

    return model

model = load_model_weights(model, ckpt['state_dict'])



Loading 589 parameters from checkpoint
Successfully loaded 588 parameters


In [15]:
# Save the model
torch.save(model.state_dict(), 'S_241214_conf-1.pth')

In [21]:
new_ckpt = torch.load(
    '/datad/pretrained/AudioDeepfakeCMs/S_241214_conf-1.pth')
new_ckpt.keys()

/tmp/ipykernel_2523835/134978345.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_ckpt = torch.load(


odict_keys(['front_end.model.mask_emb', 'front_end.model.feature_extractor.conv_layers.0.0.weight', 'front_end.model.feature_extractor.conv_layers.0.0.bias', 'front_end.model.feature_extractor.conv_layers.0.2.1.weight', 'front_end.model.feature_extractor.conv_layers.0.2.1.bias', 'front_end.model.feature_extractor.conv_layers.1.0.weight', 'front_end.model.feature_extractor.conv_layers.1.0.bias', 'front_end.model.feature_extractor.conv_layers.1.2.1.weight', 'front_end.model.feature_extractor.conv_layers.1.2.1.bias', 'front_end.model.feature_extractor.conv_layers.2.0.weight', 'front_end.model.feature_extractor.conv_layers.2.0.bias', 'front_end.model.feature_extractor.conv_layers.2.2.1.weight', 'front_end.model.feature_extractor.conv_layers.2.2.1.bias', 'front_end.model.feature_extractor.conv_layers.3.0.weight', 'front_end.model.feature_extractor.conv_layers.3.0.bias', 'front_end.model.feature_extractor.conv_layers.3.2.1.weight', 'front_end.model.feature_extractor.conv_layers.3.2.1.bias', 

In [19]:
model.load_state_dict(new_ckpt)

/tmp/ipykernel_2523835/1426476301.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(


<All keys matched successfully>

In [17]:
model

Model(
  (front_end): SSLModel(
    (model): Wav2Vec2Model(
      (feature_extractor): ConvFeatureExtractionModel(
        (conv_layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (1-4): 4 x Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (5-6): 2 x Sequential(
            (0): Conv1d(512, 512, kernel_size=